In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import os

In [2]:
pd.set_option('display.max_columns',300)

In [3]:
dados = []
for elem in os.listdir('json'):
    if 'json' in elem:
        dados.append(json.loads(open(os.path.join('json', elem),encoding='utf-8').read()) )
                 

In [11]:
dados[0]['niveis'][0]

{'ID': 0,
 'cheat': False,
 'notaDificuldade': 0,
 'difuldadeApropriada': 'Atual',
 'nivelTedioso': 5,
 'nivelDivertido': 5,
 'nivelFrustrante': 5,
 'tempoTotal': 75.5594253540039,
 'quantidadeMortes': 0,
 'dadosAntesMorrer': [{'tempoParado': 15.775725364685059,
   'tempoMovimentando': 59.78394317626953,
   'tempoAtirando': 6.659204006195068,
   'tempoRecarregando': 1.661953330039978,
   'tempoSliding': 3.818488121032715,
   'tempoDash': 4.545681953430176,
   'tempoPulando': 17.639755249023438,
   'tempoCaindo': 17.412479400634766,
   'tempoAgachado': 2.8648321628570557,
   'quantidadeTiros': 51,
   'quantidadeTirosAcertados': 31,
   'quantidadeTirosAcertadosCausandoDano': 16,
   'quantidadeTirosAcertadosSemCausarDano': 15,
   'quantidadeTirosErrados': 21,
   'quantidadeAgachado': 7,
   'quantidadePulos': 62,
   'quantidadeDash': 11,
   'quantidadeReloadApertando': 1,
   'quantidadeReloadAuto': 0,
   'distanciaPercorrida': 328.6721496582031,
   'posicoes': [{'x': 11.199999809265137, 'y

In [4]:
colunas_nivel = ['tempoParado',
    'tempoMovimentando',
    'tempoAtirando',
    'tempoRecarregando',
    'tempoSliding',
    'tempoDash',
    'tempoPulando',
    'tempoCaindo',
    'tempoAgachado',
    'quantidadeTiros',
    'quantidadeTirosAcertados',
    'quantidadeTirosAcertadosCausandoDano',
    'quantidadeTirosAcertadosSemCausarDano',
    'quantidadeTirosErrados',
    'quantidadeAgachado',
    'quantidadePulos',
    'quantidadeDash',
    'quantidadeReloadApertando',
    'quantidadeReloadAuto',
    'distanciaPercorrida']

In [16]:
def parseiaNivelColuna(dados_json):

    dict_compl =  {}
    
    for nivel in dados_json['niveis']:
        dict_nivel = {}
        id_nivel = str(nivel['ID'])
        usou_cheat = [ nivel['cheat'] ] 
        
        
        
        dict_nivel['usou_cheat_'+ id_nivel] = usou_cheat
        dict_nivel['nota_dificuldade_' + id_nivel] = [nivel['notaDificuldade']]
        dict_nivel['dificuldade_apropriada_' + id_nivel] = [nivel['difuldadeApropriada']]
        dict_nivel['nivelTedioso_' + id_nivel] = [nivel['nivelTedioso']]
        dict_nivel['nivelDivertido_' + id_nivel] = [nivel['nivelDivertido']]
        dict_nivel['nivelFrustrante_' + id_nivel] = [nivel['nivelFrustrante']]
        dict_nivel['tempoTotal_' + id_nivel] = [nivel['tempoTotal']]
        dict_nivel['quantidadeMortes_' + id_nivel ] = [nivel['quantidadeMortes']]
         
        if not usou_cheat:
            #print(str(nivel['cheat']) + id_nivel)
            for column in colunas_nivel:
                #print(nivel['dadosAntesMorrer'][-1][column])
                dict_nivel[column + '_' + 'nvl_' + id_nivel] = [ nivel['dadosAntesMorrer'][-1][column] ]
        else:
            """
            encontra a melhor tentativa , a melhor tentativa foi 
            definida nos casos em que a pessoa não consegue passar
            como aquela em que o jogador conseguiu derrotar o maior número de inimigos.
            """
            
            melhor_tentativa = 0
            melhor_ini_derrotados = 0
            for i,tentativa in enumerate(nivel['dadosAntesMorrer']):
                if (len(tentativa['inimigosDerrotados']) > melhor_ini_derrotados):
                    melhor_ini_derrotados = len(tentativa['inimigosDerrotados'])
                    melhor_tentativa = i

            for column in colunas_nivel:
                dict_nivel[column + '_' + 'nvl_' + id_nivel] = [ nivel['dadosAntesMorrer'][melhor_tentativa][column] ] 

        #print(dict_nivel)
        dict_compl = {**dict_compl , **dict_nivel}

    #print(dict_compl)
    df_nivel = pd.DataFrame(dict_compl)
    
    return df_nivel

In [17]:
def parserDados(data):
    """ Recebe o json dos dados do teste e cria um dataFrame com todos os dados."""
    df  = []
    data.keys()
    #print ({key : [value] for key,value in (data['questionarioInicial']).items() })
    
    df_quest_inic = pd.DataFrame({'inicial_' + key : [value] for key,value in (data['questionarioInicial']).items() } )
    df_quest_final = pd.DataFrame({'final_' + key : [value] for key,value in (data['questionarioFinal']).items() } )
    df_nivel = parseiaNivelColuna(data)
    
    
    
    
    
    
    df = pd.concat([df_quest_inic, df_quest_final, df_nivel], sort=True, axis=1)
    return df


dados_experimento = []
#print(type(dados_experimento))
for dado in dados:
    df_aux = parserDados(dado)  
    dados_experimento.append(df_aux)
    

df_final = pd.concat(dados_experimento).reset_index(drop=True)



In [18]:
df_final.head(3)

,inicial_idade,inicial_sexo,inicial_q3,inicial_q4,inicial_q5,inicial_q6,inicial_q7,inicial_q8,inicial_q9_a,inicial_q9_b,inicial_q9_c,inicial_q9_d,final_q1_a,final_q1_b,final_q1_c,final_q1_d,final_q1_e,final_q1_f,final_q2,final_q3_a,final_q3_b,final_q3_c,final_q4_a,final_q4_b,final_q4_c,final_q4_d,final_q4_e,final_q4_f,final_q5_a,final_q5_b,final_q5_c,final_q5_d,final_q5_e,final_q5_f,final_q5_g,final_q5_h,usou_cheat_0,nota_dificuldade_0,dificuldade_apropriada_0,nivelTedioso_0,nivelDivertido_0,nivelFrustrante_0,tempoTotal_0,quantidadeMortes_0,tempoParado_nvl_0,tempoMovimentando_nvl_0,tempoAtirando_nvl_0,tempoRecarregando_nvl_0,tempoSliding_nvl_0,tempoDash_nvl_0,tempoPulando_nvl_0,tempoCaindo_nvl_0,tempoAgachado_nvl_0,quantidadeTiros_nvl_0,quantidadeTirosAcertados_nvl_0,quantidadeTirosAcertadosCausandoDano_nvl_0,quantidadeTirosAcertadosSemCausarDano_nvl_0,quantidadeTirosErrados_nvl_0,quantidadeAgachado_nvl_0,quantidadePulos_nvl_0,quantidadeDash_nvl_0,quantidadeReloadApertando_nvl_0,quantidadeReloadAuto_nvl_0,distanciaPercorrida_nvl_0,usou_cheat_1,nota_dificuldade_1,dificuldade_apropriada_1,nivelTedioso_1,nivelDivertido_1,nivelFrustrante_1,tempoTotal_1,quantidadeMortes_1,tempoParado_nvl_1,tempoMovimentando_nvl_1,tempoAtirando_nvl_1,tempoRecarregando_nvl_1,tempoSliding_nvl_1,tempoDash_nvl_1,tempoPulando_nvl_1,tempoCaindo_nvl_1,tempoAgachado_nvl_1,quantidadeTiros_nvl_1,quantidadeTirosAcertados_nvl_1,quantidadeTirosAcertadosCausandoDano_nvl_1,quantidadeTirosAcertadosSemCausarDano_nvl_1,quantidadeTirosErrados_nvl_1,quantidadeAgachado_nvl_1,quantidadePulos_nvl_1,quantidadeDash_nvl_1,quantidadeReloadApertando_nvl_1,quantidadeReloadAuto_nvl_1,distanciaPercorrida_nvl_1,usou_cheat_2,nota_dificuldade_2,dificuldade_apropriada_2,nivelTedioso_2,nivelDivertido_2,nivelFrustrante_2,tempoTotal_2,quantidadeMortes_2,tempoParado_nvl_2,tempoMovimentando_nvl_2,tempoAtirando_nvl_2,tempoRecarregando_nvl_2,tempoSliding_nvl_2,tempoDash_nvl_2,tempoPulando_nvl_2,tempoCaindo_nvl_2,tempoAgachado_nvl_2,quantidadeTiros_nvl_2,quantidadeTirosAcertados_nvl_2,quantidadeTirosAcertadosCausandoDano_nvl_2,quantidadeTirosAcertadosSemCausarDano_nvl_2,quantidadeTirosErrados_nvl_2,quantidadeAgachado_nvl_2,quantidadePulos_nvl_2,quantidadeDash_nvl_2,quantidadeReloadApertando_nvl_2,quantidadeReloadAuto_nvl_2,distanciaPercorrida_nvl_2,usou_cheat_3,nota_dificuldade_3,dificuldade_apropriada_3,nivelTedioso_3,nivelDivertido_3,nivelFrustrante_3,tempoTotal_3,quantidadeMortes_3,tempoParado_nvl_3,tempoMovimentando_nvl_3,tempoAtirando_nvl_3,tempoRecarregando_nvl_3,tempoSliding_nvl_3,tempoDash_nvl_3,tempoPulando_nvl_3,tempoCaindo_nvl_3,tempoAgachado_nvl_3,quantidadeTiros_nvl_3,quantidadeTirosAcertados_nvl_3,quantidadeTirosAcertadosCausandoDano_nvl_3,quantidadeTirosAcertadosSemCausarDano_nvl_3,quantidadeTirosErrados_nvl_3,quantidadeAgachado_nvl_3,quantidadePulos_nvl_3,quantidadeDash_nvl_3,quantidadeReloadApertando_nvl_3,quantidadeReloadAuto_nvl_3,distanciaPercorrida_nvl_3,usou_cheat_4,nota_dificuldade_4,dificuldade_apropriada_4,nivelTedioso_4,nivelDivertido_4,nivelFrustrante_4,tempoTotal_4,quantidadeMortes_4,tempoParado_nvl_4,tempoMovimentando_nvl_4,tempoAtirando_nvl_4,tempoRecarregando_nvl_4,tempoSliding_nvl_4,tempoDash_nvl_4,tempoPulando_nvl_4,tempoCaindo_nvl_4,tempoAgachado_nvl_4,quantidadeTiros_nvl_4,quantidadeTirosAcertados_nvl_4,quantidadeTirosAcertadosCausandoDano_nvl_4,quantidadeTirosAcertadosSemCausarDano_nvl_4,quantidadeTirosErrados_nvl_4,quantidadeAgachado_nvl_4,quantidadePulos_nvl_4,quantidadeDash_nvl_4,quantidadeReloadApertando_nvl_4,quantidadeReloadAuto_nvl_4,distanciaPercorrida_nvl_4,usou_cheat_5,nota_dificuldade_5,dificuldade_apropriada_5,nivelTedioso_5,nivelDivertido_5,nivelFrustrante_5,tempoTotal_5,quantidadeMortes_5,tempoParado_nvl_5,tempoMovimentando_nvl_5,tempoAtirando_nvl_5,tempoRecarregando_nvl_5,tempoSliding_nvl_5,tempoDash_nvl_5,tempoPulando_nvl_5,tempoCaindo_nvl_5,tempoAgachado_nvl_5,quantidadeTiros_nvl_5,quantidadeTirosAcertados_nvl_5,quantidadeTirosAcertadosCau

In [20]:
df_final.nota_dificuldade_6

0      7
1      9
2      7
3     10
4      7
5      7
6      5
7      7
8      5
9      4
10     7
11     6
12     9
13     8
14     3
15     8
16     4
17     6
18     8
Name: nota_dificuldade_6, dtype: int64